In [4]:
import sys
import random
import numpy as np
from numpy import array,zeros,dot
from numpy import genfromtxt

In [6]:
data = genfromtxt('mnist_train.csv', delimiter=',')

In [12]:
class NN:
    def __init__(self, inp_unit, out_unit,  hidden_shape, activation, cost, w_upd,itr=1000, alpha=0.01, momentum=0.9, decay=0.1):
        self.inp_unit,self.out_unit,self.hidden_shape,self.itr,self.alpha,self.momentum,self.decay,self.activation = inp_unit,\
                                                            out_unit, hidden_shape, itr, alpha, momentum, decay, activation 
        self.cost,self.w_upd = cost,w_upd
        self.flag,self.W,self.B,self.arc_dim = True,0,0,0     
        self.pred,self.h_layer,self.h_layer_A,self.temp,self.Frist_time,self._v,self._m = 0,0,0,0,0,0,0
        self.testing = []
    
    def initz(self,inp_unit,out_unit):
        self.W,self.B,self.arc_dim = 0,0,0
        self.pred,self.temp,self.Frist_time,self._v,self._m = 0,0,True,zeros((len(self.hidden_shape)+1))\
                                                                            ,zeros((len(self.hidden_shape)+1))
        if len(inp_unit.shape) > 1 and len(out_unit.shape) > 1:
            self.W,self.B,self.arc_dim = [],[],[inp_unit.shape[1]] + self.hidden_shape + [out_unit.shape[1]]
        else:
            self.flag = False
    
    def anneal_alpha(self):
        self.alpha = self.alpha/2
    
    def sigmoid(self,x,flag):
        x[x < -100] = -100
        return 1 / (1.0 + np.exp(-x)) if flag == 0 else np.exp(-x)/((1+np.exp(-x))**2)

    def tanh(self,x,flag):
        x[x < -100] = -100
        return np.tanh(x) if flag == 0 else 1.0 - np.tanh(x)**2.0 

    def random_weight(self,shp):
        return np.random.normal(loc = 0, scale = 0.3, size = shp)
        
    def _feed_forward(self,A):
        self.h_layer,self.h_layer_A = [],[]
        if self.Frist_time is True:
            for idx in range(len(self.arc_dim)-1):
                self.W.append(self.random_weight([self.arc_dim[idx],self.arc_dim[idx+1]]))
                self.B.append(self.random_weight([1,self.arc_dim[idx+1]]))
            self.Frist_time = False
        for idx in range(len(self.arc_dim)-2):
            if self.activation[idx] is "sigmoid":
                A = dot(A,self.W[idx]) + self.B[idx]
                self.h_layer.append(A)
                C = self.sigmoid(A,0)
                self.h_layer_A.append(C)
            else:
                A = dot(A,self.W[idx]) + self.B[idx]
                self.h_layer.append(A)
                C = self.tanh(A,0)
                self.h_layer_A.append(C)
        
        if self.activation[-1] is "sigmoid":
            self.temp = dot(C,self.W[idx+1]) + self.B[idx+1]
            self.pred = self.sigmoid(self.temp,0) 
        else:
            self.temp = dot(C,self.W[idx+1]) + self.B[idx+1]
            self.pred = self.tanh(self.temp,0)
    
    def cost_func(self,targets,predct,flg=0):
        if flg is 0:
            return(0.5*np.sum((self.out_unit-self.pred)**2))
        else:
            return -1*np.sum(self.out_unit*np.log(self.pred) + (1 - self.out_unit)*np.log(self.pred))
        
    def _back_prop(self,Inp):
        weight_delta = []
        if self.cost is "sqr":
            if self.activation[-1] == "sigmoid":
                delta = np.multiply(-(self.out_unit-self.pred), self.sigmoid(self.temp,1))
            else:
                delta = np.multiply(-(self.out_unit-self.pred), self.tanh(self.temp,1))
        else:                                                                                        # cross
            if self.activation[-1] == "sigmoid":
                delta = np.multiply(-(self.out_unit-self.pred)/(self.pred*(1-self.pred)), self.sigmoid(self.temp,1))
            else:
                delta = np.multiply(-(self.out_unit-self.pred)/(self.pred*(1-self.pred)), self.tanh(self.temp,1))
        weight_delta.append(np.dot(self.h_layer_A[-1].T, delta))
        for idx in reversed(range(len(self.h_layer))):
            if self.activation[idx] == "sigmoid":
                delta2 = np.dot(delta, self.W[idx+1].T)*self.sigmoid(self.h_layer[idx],1)
            else:
                delta2 = np.dot(delta, self.W[idx+1].T)*self.tanh(self.h_layer[idx],1)
            if idx > 0:
                weight_delta.append(np.dot(self.h_layer_A[idx-1].T, delta2))
            else:
                weight_delta.append(np.dot(Inp.T, delta2))
            delta = delta2        
        weight_delta.reverse()
        return weight_delta
    
    def _weight_update_SG(self,dx,alpha):
        for i in range(len(dx)):
            self.W[i] += alpha * dx[i]
    
    def _weight_update_Momentum(self,dx,alpha,mu):
        for i in range(len(dx)):
            self._v[i] = mu * self._v[i] - alpha * dx[i]
            self.W[i] += self._v[i]
            
    def _weight_update_Nag(self,dx,alpha,mu):
        for i in range(len(dx)):
            v_prev = self._v[i]
            self._v[i] = mu * self._v[i] - alpha * dx[i]
            self.W[i] += -mu * v_prev + (1 + mu) * self._v[i]
            
    def _weight_update_Adam(self,dx,alpha,t,eps=1e-8,beta1=0.9,beta2=0.999):
        for i in range(len(dx)):
            self._m[i] = beta1*self._m[i] + (1-beta1)*dx[i]
            mt = self._m[i] / (1-beta1**t)
            self._v[i] = beta2*self._v[i] + (1-beta2)*(dx[i]**2)
            vt = self._v[i] / (1-beta2**t)
            self.W[i] += - alpha * mt / (np.sqrt(vt) + eps)
    
    def mini_batch(self,batch_size = 25):
        batch_size,num = 25,self.inp_unit.shape[0] / batch_size
        return zip(np.array_split(self.inp_unit, num, axis=0),np.array_split(self.out_unit, num, axis=0))
    
    def error(self,x,y):
        err = 0
        for i in range(x.shape[0]):
            if (np.argmax(x[i]) == np.argmax(y[i])):
                 err+=1.0
        return err/x.shape[0]
    
    def train(self,num_itr=100,v_x,v_y,batch_sz):
        self.initz(self.inp_unit,self.out_unit)
        batch = self.mini_batch(batch_sz)
        err,loss = [],[]
        v_err,v_loss = [],[]
        for itr in range(num_epoch):
            step = 0
            for X, Y in batch:
                self._feed_forward(X)
                dw = self._back_prop(X)
                if self.w_upd is 'sg':
                    self._weight_update_SG(dw,self.alpha)
                elif self.w_upd is 'mu':
                    self._weight_update_Momentum(dw,self.alpha,self.momentum)
                elif self.w_upd is 'nag':
                    self._weight_update_Nag(dw,self.alpha,self.momentum)
                else:
                    self._weight_update_Adam(dw,self.alpha,itr)
                step += 1
                if step%100 == 0:
                    loss.append(self.cost_func(X,Y))
                    err.append(self.error(X,Y))
                    self._feed_forward(v_x)
                    loss.append(self.cost_func(v_x,v_y))
                    err.append(self.error(v_x,v_y))
        return err,loss,v_err,v_loss
            
#         for i in range(1):
#             self._feed_forward(self.inp_unit)
#             dx = self._back_prop(self.inp_unit)
#             self._weight_update_SG(dx,self.alpha)
#             if(i%10 == 0):
#                 print(self.cost_func(self.out_unit,self.pred,0))

In [20]:
# nn = NN(train_x,train_y_1,[10,15,20],['tanh','sigmoid','tanh','sigmoid'],'sqr','sg')
# nn.train(1000,valid_x,valid_y_1)

In [48]:
# a = -300
# print((np.exp(-a)/((1+np.exp(-a))**2)))
a = -100
print((np.exp(-a)/((1+np.exp(-a))**2)))

3.72007597602e-44


In [30]:
# np.finfo(a.dtype)
print(np.finfo(a).maxexp,np.finfo(a).minexp)

1024 -1022


In [10]:
a = array([[1,2,3,4,5,6],[1,2,3,4,6,5]])
np.sum(a)

42

In [18]:
a = array([1,2,3,45,6,7,8])
b = array([0,0,0,1,0,0,0])
print(np.argmax(a) == np.argmax(b))

True


In [19]:
a = 'hello'
a is 'hello'

True

# Preprocessing

In [8]:
train = data[:50000]
valid = data[50000:]

In [9]:
train_y = train[:,0]
train_x = train[:,1:]
train_y_1 = np.eye(10)[train_y.astype(int)]
valid_y = valid[:,0]
valid_x = valid[:,1:]
valid_y_1 = np.eye(10)[valid_y.astype(int)]

In [5]:
import pickle

In [216]:
pickle.dump(train_x,open("train_x.pkl",'wb'))
pickle.dump(train_y_1,open("train_y.pkl",'wb'))
pickle.dump(valid_x,open("valid_x.pkl",'wb'))
pickle.dump(valid_y_1,open("valid_y.pkl",'wb'))

# Testing Tensorflow

In [133]:
import tensorflow as tf

In [163]:
x = tf.placeholder(tf.float32, [None, 5])
y = tf.placeholder(tf.float32, [None, 2])
w1 = tf.placeholder(tf.float32, [5, 3])
w2 = tf.placeholder(tf.float32, [3, 4])
w3 = tf.placeholder(tf.float32, [4, 5])
w4 = tf.placeholder(tf.float32, [5, 2])
b1 = tf.placeholder(tf.float32, [1, 3])
b2 = tf.placeholder(tf.float32, [1, 4])
b3 = tf.placeholder(tf.float32, [1, 5])
b4 = tf.placeholder(tf.float32, [1, 2])

In [164]:
g1 = tf.tanh(tf.add(tf.matmul(x, w1), b1))
print(g1)
g2 = tf.sigmoid(tf.add(tf.matmul(g1, w2), b2))
print(g2)
g3 = tf.tanh(tf.add(tf.matmul(g2, w3), b3))
print(g3)
g4 = (tf.add(tf.matmul(g3, w4), b4))
print(g4)

Tensor("Tanh_17:0", shape=(?, 3), dtype=float32)
Tensor("Sigmoid_9:0", shape=(?, 4), dtype=float32)
Tensor("Tanh_18:0", shape=(?, 5), dtype=float32)
Tensor("Add_35:0", shape=(?, 2), dtype=float32)


In [165]:
with tf.Session() as sess:
    f = sess.run(g4,feed_dict = {x: a, y: b, w1: nn.W[0], w2: nn.W[1], w3: nn.W[2], w4: nn.W[3],
                             b1: nn.B[0], b2: nn.B[1], b3: nn.B[2], b4: nn.B[3]})

In [166]:
f

array([[-0.12157428, -0.07123873],
       [-0.11776307, -0.07139242],
       [-0.13612098, -0.07489069]], dtype=float32)

In [8]:
array([[1,2],[2,3]]) - array([[3,1],[1,2]])

array([[-2,  1],
       [ 1,  1]])